In [3]:
import pandas as pd
import numpy as np
from prophet import Prophet
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 1. Xử lý và lưu dữ liệu MBB
# Đọc dữ liệu MBB
data_day_mbb = pd.read_csv('MBB.csv')
data_day_mbb['Ngày'] = pd.to_datetime(data_day_mbb['Ngày'], format='%d/%m/%Y')
data_day_mbb = data_day_mbb[(data_day_mbb['Ngày'] >= pd.to_datetime('2020-04-10')) & 
                            (data_day_mbb['Ngày'] <= pd.to_datetime('2025-04-10'))]
data_day_mbb['Lần cuối'] = data_day_mbb['Lần cuối'].astype(str).str.replace(',', '').astype(float)

# Tạo dữ liệu đầu tháng cho MBB
data_month_mbb = data_day_mbb.groupby(pd.Grouper(key='Ngày', freq='MS'))['Lần cuối'].first().reset_index()
data_month_mbb.columns = ['Date', 'MBB_Price']

# Đọc dữ liệu S&P 500 và VNINDEX
sp500_data = pd.read_csv('SP500_Full_Predictions_2015_2028.csv')
sp500_data['Date'] = pd.to_datetime(sp500_data['Date'])
vnindex_data = pd.read_csv('VNIINDEX_Full_Predictions_2015_2028.csv')
vnindex_data['Date'] = pd.to_datetime(vnindex_data['Date'])

# Tạo dữ liệu đầu tháng cho S&P 500 và VNINDEX
sp500_month = sp500_data.groupby(pd.Grouper(key='Date', freq='MS'))['Price'].first().reset_index()
vnindex_month = vnindex_data.groupby(pd.Grouper(key='Date', freq='MS'))['Lần cuối'].first().reset_index()

# Ghép dữ liệu tháng
data_month_mbb = data_month_mbb.merge(sp500_month, on='Date', how='left')
data_month_mbb = data_month_mbb.merge(vnindex_month, on='Date', how='left')
data_month_mbb = data_month_mbb.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
data_month_mbb = data_month_mbb.dropna()

# Ghép dữ liệu ngày
data_day_mbb = data_day_mbb.rename(columns={'Ngày': 'Date', 'Lần cuối': 'MBB_Price'})
data_day_mbb = data_day_mbb.merge(sp500_data[['Date', 'Price']], on='Date', how='left')
data_day_mbb = data_day_mbb.merge(vnindex_data[['Date', 'Lần cuối']], on='Date', how='left')
data_day_mbb = data_day_mbb.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
data_day_mbb = data_day_mbb.dropna()

# Lưu dữ liệu đã xử lý
data_month_mbb.to_csv('MBB_Monthly_Processed.csv', index=False)
data_day_mbb.to_csv('MBB_Daily_Processed.csv', index=False)
print("Dữ liệu đã được xử lý và lưu vào 'MBB_Monthly_Processed.csv' và 'MBB_Daily_Processed.csv'")

# 2. Huấn luyện mô hình Prophet để dự đoán giá đầu tháng của MBB
model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
model_prophet.add_regressor('SP500_Price')
model_prophet.add_regressor('VNINDEX_Price')
model_prophet.fit(data_month_mbb.rename(columns={'Date': 'ds', 'MBB_Price': 'y'}))

# Tạo khung thời gian tương lai (từ 1/5/2025 đến 1/5/2028)
future_dates_month = pd.date_range(start='2025-05-01', end='2028-05-01', freq='MS')
future_df_month = pd.DataFrame({'ds': future_dates_month})

# Ghép giá dự đoán đầu tháng của S&P 500 và VNINDEX
future_df_month = future_df_month.merge(sp500_month[sp500_month['Date'] >= '2025-05-01'], 
                                       left_on='ds', right_on='Date', how='left')
future_df_month = future_df_month.merge(vnindex_month[vnindex_month['Date'] >= '2025-05-01'], 
                                       left_on='ds', right_on='Date', how='left')
future_df_month = future_df_month.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
future_df_month = future_df_month[['ds', 'SP500_Price', 'VNINDEX_Price']].fillna(method='ffill')

# Dự đoán giá đầu tháng của MBB
forecast = model_prophet.predict(future_df_month)
predicted_monthly_mbb = forecast[['ds', 'yhat']]
predicted_monthly_mbb.columns = ['Date', 'Predicted_Price']

# 3. Huấn luyện mô hình LSTM để dự đoán giá ngày
scaler = MinMaxScaler()
scaled_data_day = scaler.fit_transform(data_day_mbb[['MBB_Price', 'SP500_Price', 'VNINDEX_Price']])
if np.any(np.isnan(scaled_data_day)) or np.any(np.isinf(scaled_data_day)):
    scaled_data_day = np.nan_to_num(scaled_data_day, nan=0.0, posinf=0.0, neginf=0.0)

time_step = 300
def create_lstm_data(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X, y = create_lstm_data(scaled_data_day, time_step)

X_train_tensor = torch.FloatTensor(X)
y_train_tensor = torch.FloatTensor(y).reshape(-1, 1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, input_size=3, hidden_size=50, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model_lstm = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_lstm = model_lstm.to(device)

num_epochs = 30
for epoch in range(num_epochs):
    model_lstm.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model_lstm(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model_lstm.parameters(), max_norm=1.0)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

# 4. Dự đoán giá ngày từ 10/4/2025 đến 1/5/2028 với scale liên tục
historical_data = scaled_data_day.tolist()
future_dates_daily = pd.date_range(start='2025-04-10', end='2028-05-01', freq='D')
daily_df = pd.DataFrame({'Date': future_dates_daily})

# Ghép giá dự đoán S&P 500 và VNINDEX cho ngày tương lai
daily_df = daily_df.merge(sp500_data[['Date', 'Price']], on='Date', how='left')
daily_df = daily_df.merge(vnindex_data[['Date', 'Lần cuối']], on='Date', how='left')
daily_df = daily_df.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
daily_df = daily_df[['Date', 'SP500_Price', 'VNINDEX_Price']].fillna(method='ffill')

# Chuẩn hóa dữ liệu dự đoán
scaled_future_data = scaler.transform(daily_df[['SP500_Price', 'VNINDEX_Price']])
scaled_future_data = np.hstack((np.zeros((len(scaled_future_data), 1)), scaled_future_data))  # Placeholder cho giá MBB

model_lstm.eval()
predictions_daily = []
current_window = historical_data[-time_step:]
last_scaled_price = scaler.inverse_transform([historical_data[-1]])[0, 0]

i = 0
for j in range(len(predicted_monthly_mbb) - 1):
    start_date = max(predicted_monthly_mbb['Date'].iloc[j], pd.to_datetime('2025-04-10'))
    end_date = predicted_monthly_mbb['Date'].iloc[j + 1] - pd.Timedelta(days=1)
    next_month_start = predicted_monthly_mbb['Date'].iloc[j + 1]
    if j == len(predicted_monthly_mbb) - 2:
        end_date = pd.to_datetime('2028-05-01')

    if start_date > end_date:
        continue

    segment_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    n_days = len(segment_dates)
    if n_days == 0:
        continue

    segment_predictions = []
    current_window_tensor = torch.FloatTensor(current_window[-time_step:]).unsqueeze(0).to(device)
    with torch.no_grad():
        window = current_window_tensor
        for day in range(n_days):
            pred = model_lstm(window).cpu().numpy()[0, 0]
            segment_predictions.append(pred)
            new_window = np.append(window.cpu().numpy()[0, 1:, :], 
                                  [[pred, scaled_future_data[i + day, 1], scaled_future_data[i + day, 2]]], axis=0)
            window = torch.FloatTensor(new_window).unsqueeze(0).to(device)

    i += n_days

    # Đảo ngược chuẩn hóa
    segment_predictions = np.array(segment_predictions).reshape(-1, 1)
    segment_predictions_rescaled = scaler.inverse_transform(
        np.hstack((segment_predictions, scaled_future_data[i - n_days:i, 1:]))
    )[:, 0]

    # Scale giá ngày để khớp với giá đầu tháng
    if end_date >= next_month_start - pd.Timedelta(days=1):
        monthly_price_next = predicted_monthly_mbb['Predicted_Price'].iloc[j + 1]
        daily_price_at_end = segment_predictions_rescaled[-1]
        daily_price_at_start = segment_predictions_rescaled[0]

        adjustment_at_start = last_scaled_price - daily_price_at_start
        adjustment_at_end = monthly_price_next - daily_price_at_end
        segment_length = n_days
        adjustments = np.linspace(adjustment_at_start, adjustment_at_end, segment_length)
        segment_predictions_scaled = segment_predictions_rescaled + adjustments
        last_scaled_price = segment_predictions_scaled[-1]
    else:
        adjustment_at_start = last_scaled_price - daily_price_at_start
        segment_predictions_scaled = segment_predictions_rescaled + adjustment_at_start
        last_scaled_price = segment_predictions_scaled[-1]

    # Cập nhật current_window
    scaled_segment = scaler.transform(
        np.column_stack((segment_predictions_scaled.reshape(-1, 1), scaled_future_data[i - n_days:i, 1:]))
    )
    current_window.extend(scaled_segment.tolist())
    predictions_daily.extend(segment_predictions_scaled.tolist())

# 5. Tạo DataFrame kết hợp dữ liệu lịch sử và dự đoán
combined_data = pd.concat([data_day_mbb[data_day_mbb['Date'] <= pd.to_datetime('2025-04-10')], 
                          pd.DataFrame({'Date': future_dates_daily, 'MBB_Price': predictions_daily})])
combined_data = combined_data[(combined_data['Date'] >= pd.to_datetime('2020-04-10')) & 
                              (combined_data['Date'] <= pd.to_datetime('2028-05-01'))]
combined_data = combined_data.reset_index(drop=True)

# Lưu kết quả vào file CSV
combined_data.to_csv('MBB_Full_Predictions_2020_2028.csv', index=False)
print("Dữ liệu đầy đủ từ 10/4/2020 đến 1/5/2028 đã được lưu vào 'MBB_Full_Predictions_2020_2028.csv'")

# Biểu đồ tổng quan từ 10/4/2020 đến 1/5/2028
plt.figure(figsize=(14, 7))
plt.plot(data_month_mbb['Date'], data_month_mbb['MBB_Price'], label='Dữ liệu lịch sử (Tháng)', color='blue')
plt.plot(data_day_mbb['Date'], data_day_mbb['MBB_Price'], label='Dữ liệu lịch sử (Ngày)', color='lightblue', alpha=0.5)

predicted_monthly_mbb_filtered = predicted_monthly_mbb[predicted_monthly_mbb['Date'] >= pd.to_datetime('2025-04-10')]
plt.plot(predicted_monthly_mbb_filtered['Date'], predicted_monthly_mbb_filtered['Predicted_Price'], 
         label='Dự đoán (Tháng)', color='orange', marker='o')

plt.plot(combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['Date'], 
         combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['MBB_Price'], 
         label='Dự đoán (Ngày, đã scale)', color='green', linestyle='--')
plt.title('Dự đoán MBB: Tháng và Ngày (Scale liên tục)')
plt.xlabel('Ngày')
plt.ylabel('Giá MBB')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('MBB_Prediction_Chart_Full.png')
plt.close()

# Biểu đồ chi tiết từ 10/4/2025 đến 1/5/2028
mask_detail = (combined_data['Date'] >= pd.to_datetime('2025-04-10')) & (combined_data['Date'] <= pd.to_datetime('2028-05-01'))
combined_data_detail = combined_data[mask_detail]
month_df_detail = predicted_monthly_mbb[
    (predicted_monthly_mbb['Date'] >= pd.to_datetime('2025-04-10')) & 
    (predicted_monthly_mbb['Date'] <= pd.to_datetime('2028-05-01'))
]

plt.figure(figsize=(14, 7))
plt.plot(month_df_detail['Date'], month_df_detail['Predicted_Price'], label='Dự đoán (Tháng)', color='orange', marker='o')
plt.plot(combined_data_detail['Date'], combined_data_detail['MBB_Price'], label='Dự đoán (Ngày, đã scale)', color='green', linestyle='--')
plt.title('Dự đoán MBB: Tháng và Ngày (10/4/2025 - 1/5/2028)')
plt.xlabel('Ngày')
plt.ylabel('Giá MBB')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('MBB_Prediction_Chart_Detail_2025_2028.png')
plt.close()

print("Biểu đồ tổng quan đã được lưu vào 'MBB_Prediction_Chart_Full.png'")
print("Biểu đồ chi tiết (10/4/2025 - 1/5/2028) đã được lưu vào 'MBB_Prediction_Chart_Detail_2025_2028.png'")

18:26:06 - cmdstanpy - INFO - Chain [1] start processing


Dữ liệu đã được xử lý và lưu vào 'MBB_Monthly_Processed.csv' và 'MBB_Daily_Processed.csv'


18:26:06 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Tuan Anh\AppData\Local\Temp\ipykernel_23216\629928690.py:66: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  future_df_month = future_df_month[['ds', 'SP500_Price', 'VNINDEX_Price']].fillna(method='ffill')


Epoch [1/30], Loss: 0.023791
Epoch [2/30], Loss: 0.019156
Epoch [3/30], Loss: 0.003091
Epoch [4/30], Loss: 0.006628
Epoch [5/30], Loss: 0.004883
Epoch [6/30], Loss: 0.001855
Epoch [7/30], Loss: 0.002059
Epoch [8/30], Loss: 0.000711
Epoch [9/30], Loss: 0.001437
Epoch [10/30], Loss: 0.001122
Epoch [11/30], Loss: 0.002157
Epoch [12/30], Loss: 0.002861
Epoch [13/30], Loss: 0.001179
Epoch [14/30], Loss: 0.001453
Epoch [15/30], Loss: 0.000763
Epoch [16/30], Loss: 0.001355
Epoch [17/30], Loss: 0.001319
Epoch [18/30], Loss: 0.001826
Epoch [19/30], Loss: 0.003489
Epoch [20/30], Loss: 0.002049
Epoch [21/30], Loss: 0.000933
Epoch [22/30], Loss: 0.001203
Epoch [23/30], Loss: 0.001175
Epoch [24/30], Loss: 0.002511
Epoch [25/30], Loss: 0.002300
Epoch [26/30], Loss: 0.001255
Epoch [27/30], Loss: 0.002214
Epoch [28/30], Loss: 0.001843
Epoch [29/30], Loss: 0.002011
Epoch [30/30], Loss: 0.000868


C:\Users\Tuan Anh\AppData\Local\Temp\ipykernel_23216\629928690.py:137: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily_df = daily_df[['Date', 'SP500_Price', 'VNINDEX_Price']].fillna(method='ffill')


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- MBB_Price


In [6]:
import pandas as pd
import numpy as np
from prophet import Prophet
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 1. Xử lý và lưu dữ liệu MBB
# Đọc dữ liệu MBB
data_day_mbb = pd.read_csv('MBB.csv')
data_day_mbb['Ngày'] = pd.to_datetime(data_day_mbb['Ngày'], format='%d/%m/%Y')
data_day_mbb = data_day_mbb[(data_day_mbb['Ngày'] >= pd.to_datetime('2020-04-10')) & 
                            (data_day_mbb['Ngày'] <= pd.to_datetime('2025-04-10'))]
data_day_mbb['Lần cuối'] = data_day_mbb['Lần cuối'].astype(str).str.replace(',', '').astype(float)

# Tạo dữ liệu đầu tháng cho MBB
data_month_mbb = data_day_mbb.groupby(pd.Grouper(key='Ngày', freq='MS'))['Lần cuối'].first().reset_index()
data_month_mbb.columns = ['Date', 'MBB_Price']

# Đọc dữ liệu S&P 500 và VNINDEX
sp500_data = pd.read_csv('SP500_Full_Predictions_2015_2028.csv')
sp500_data['Date'] = pd.to_datetime(sp500_data['Date'])
vnindex_data = pd.read_csv('VNIINDEX_Full_Predictions_2015_2028.csv')
vnindex_data['Date'] = pd.to_datetime(vnindex_data['Date'])

# Tạo dữ liệu đầu tháng cho S&P 500 và VNINDEX
sp500_month = sp500_data.groupby(pd.Grouper(key='Date', freq='MS'))['Price'].first().reset_index()
vnindex_month = vnindex_data.groupby(pd.Grouper(key='Date', freq='MS'))['Lần cuối'].first().reset_index()

# Ghép dữ liệu tháng
data_month_mbb = data_month_mbb.merge(sp500_month, on='Date', how='left')
data_month_mbb = data_month_mbb.merge(vnindex_month, on='Date', how='left')
data_month_mbb = data_month_mbb.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
data_month_mbb = data_month_mbb.dropna()

# Ghép dữ liệu ngày
data_day_mbb = data_day_mbb.rename(columns={'Ngày': 'Date', 'Lần cuối': 'MBB_Price'})
data_day_mbb = data_day_mbb.merge(sp500_data[['Date', 'Price']], on='Date', how='left')
data_day_mbb = data_day_mbb.merge(vnindex_data[['Date', 'Lần cuối']], on='Date', how='left')
data_day_mbb = data_day_mbb.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
data_day_mbb = data_day_mbb.dropna()

# Lưu dữ liệu đã xử lý
data_month_mbb.to_csv('MBB_Monthly_Processed.csv', index=False)
data_day_mbb.to_csv('MBB_Daily_Processed.csv', index=False)
print("Dữ liệu đã được xử lý và lưu vào 'MBB_Monthly_Processed.csv' và 'MBB_Daily_Processed.csv'")

# 2. Huấn luyện mô hình Prophet để dự đoán giá đầu tháng của MBB
model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
model_prophet.add_regressor('SP500_Price')
model_prophet.add_regressor('VNINDEX_Price')
model_prophet.fit(data_month_mbb.rename(columns={'Date': 'ds', 'MBB_Price': 'y'}))

# Tạo khung thời gian tương lai (từ 1/5/2025 đến 1/5/2028)
future_dates_month = pd.date_range(start='2025-05-01', end='2028-05-01', freq='MS')
future_df_month = pd.DataFrame({'ds': future_dates_month})

# Ghép giá dự đoán đầu tháng của S&P 500 và VNINDEX
future_df_month = future_df_month.merge(sp500_month[sp500_month['Date'] >= '2025-05-01'], 
                                       left_on='ds', right_on='Date', how='left')
future_df_month = future_df_month.merge(vnindex_month[vnindex_month['Date'] >= '2025-05-01'], 
                                       left_on='ds', right_on='Date', how='left')
future_df_month = future_df_month.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
future_df_month = future_df_month[['ds', 'SP500_Price', 'VNINDEX_Price']].ffill()

# Dự đoán giá đầu tháng của MBB
forecast = model_prophet.predict(future_df_month)
predicted_monthly_mbb = forecast[['ds', 'yhat']]
predicted_monthly_mbb.columns = ['Date', 'Predicted_Price']

# 3. Huấn luyện mô hình LSTM để dự đoán giá ngày
scaler = MinMaxScaler()
scaled_data_day = scaler.fit_transform(data_day_mbb[['MBB_Price', 'SP500_Price', 'VNINDEX_Price']])
if np.any(np.isnan(scaled_data_day)) or np.any(np.isinf(scaled_data_day)):
    scaled_data_day = np.nan_to_num(scaled_data_day, nan=0.0, posinf=0.0, neginf=0.0)

time_step = 300
def create_lstm_data(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X, y = create_lstm_data(scaled_data_day, time_step)

X_train_tensor = torch.FloatTensor(X)
y_train_tensor = torch.FloatTensor(y).reshape(-1, 1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, input_size=3, hidden_size=50, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model_lstm = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_lstm = model_lstm.to(device)

num_epochs = 3
for epoch in range(num_epochs):
    model_lstm.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model_lstm(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model_lstm.parameters(), max_norm=1.0)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

# 4. Dự đoán giá ngày từ 10/4/2025 đến 1/5/2028 với scale liên tục
historical_data = scaled_data_day.tolist()
future_dates_daily = pd.date_range(start='2025-04-10', end='2028-05-01', freq='D')
daily_df = pd.DataFrame({'Date': future_dates_daily})

# Ghép giá dự đoán S&P 500 và VNINDEX cho ngày tương lai
daily_df = daily_df.merge(sp500_data[['Date', 'Price']], on='Date', how='left')
daily_df = daily_df.merge(vnindex_data[['Date', 'Lần cuối']], on='Date', how='left')
daily_df = daily_df.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
daily_df = daily_df[['Date', 'SP500_Price', 'VNINDEX_Price']].ffill()

# Chuẩn hóa dữ liệu dự đoán (thêm cột placeholder cho MBB_Price)
daily_df['MBB_Price'] = 0  # Placeholder sẽ được cập nhật sau
scaled_future_data = scaler.transform(daily_df[['MBB_Price', 'SP500_Price', 'VNINDEX_Price']])

# Cắt scaled_future_data để khớp với future_dates_daily
scaled_future_data = scaled_future_data[:len(future_dates_daily)]

model_lstm.eval()
predictions_daily = []
current_window = historical_data[-time_step:]

# Lấy giá thực tế tại 10/4/2025
last_actual_price = data_day_mbb[data_day_mbb['Date'] == pd.to_datetime('2025-04-10')]['MBB_Price'].iloc[0]

# Dự đoán từng ngày
total_days = len(future_dates_daily)
for i in range(total_days):
    current_window_tensor = torch.FloatTensor(current_window[-time_step:]).unsqueeze(0).to(device)
    with torch.no_grad():
        pred = model_lstm(current_window_tensor).cpu().numpy()[0, 0]
        predictions_daily.append(pred)
        # Cập nhật cửa sổ với giá dự đoán
        new_window_data = np.append(current_window_tensor.cpu().numpy()[0, 1:, :], 
                                   [[pred, scaled_future_data[i, 1], scaled_future_data[i, 2]]], axis=0)
        current_window.append([pred, scaled_future_data[i, 1], scaled_future_data[i, 2]])

# Đảo ngược chuẩn hóa toàn bộ dự đoán
predictions_daily = np.array(predictions_daily).reshape(-1, 1)
predictions_rescaled = scaler.inverse_transform(
    np.hstack((predictions_daily, scaled_future_data[:, 1:]))
)[:, 0]

# Scale giá ngày để khớp với giá đầu tháng
adjusted_predictions = predictions_rescaled.copy()
current_segment_start_idx = 0
for j in range(len(predicted_monthly_mbb)):
    if j == 0:
        start_date = pd.to_datetime('2025-04-10')
        end_date = pd.to_datetime('2025-04-30')
        next_month_start = predicted_monthly_mbb['Date'].iloc[j]
    else:
        start_date = predicted_monthly_mbb['Date'].iloc[j - 1]
        end_date = predicted_monthly_mbb['Date'].iloc[j] - pd.Timedelta(days=1)
        next_month_start = predicted_monthly_mbb['Date'].iloc[j]

    if j == len(predicted_monthly_mbb) - 1:
        end_date = pd.to_datetime('2028-05-01')

    if start_date > end_date:
        continue

    segment_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    segment_indices = [i for i, date in enumerate(future_dates_daily) if date in segment_dates]
    if not segment_indices:
        continue

    segment_start_idx = segment_indices[0]
    segment_end_idx = segment_indices[-1]
    segment_predictions = adjusted_predictions[segment_start_idx:segment_end_idx + 1]

    if j == 0:  # Đoạn đầu tiên từ 10/4/2025 đến 30/4/2025
        monthly_price_next = predicted_monthly_mbb[predicted_monthly_mbb['Date'] == next_month_start]['Predicted_Price'].iloc[0]
        daily_price_at_start = last_actual_price  # Giá thực tế tại 10/4/2025
        daily_price_at_end = segment_predictions[-1]

        adjustment_at_start = 0  # Không điều chỉnh giá tại 10/4/2025
        adjustment_at_end = monthly_price_next - daily_price_at_end
        segment_length = len(segment_predictions)
        adjustments = np.linspace(adjustment_at_start, adjustment_at_end, segment_length)
        adjusted_predictions[segment_start_idx:segment_end_idx + 1] = segment_predictions + adjustments
    else:  # Các đoạn sau từ 1/5/2025 trở đi
        monthly_price_current = predicted_monthly_mbb[predicted_monthly_mbb['Date'] == start_date]['Predicted_Price'].iloc[0]
        if end_date >= next_month_start - pd.Timedelta(days=1):
            monthly_price_next = predicted_monthly_mbb[predicted_monthly_mbb['Date'] == next_month_start]['Predicted_Price'].iloc[0]
            daily_price_at_end = segment_predictions[-1]
            daily_price_at_start = segment_predictions[0]

            adjustment_at_start = monthly_price_current - daily_price_at_start
            adjustment_at_end = monthly_price_next - daily_price_at_end
            segment_length = len(segment_predictions)
            adjustments = np.linspace(adjustment_at_start, adjustment_at_end, segment_length)
            adjusted_predictions[segment_start_idx:segment_end_idx + 1] = segment_predictions + adjustments
        else:
            adjustment_at_start = monthly_price_current - segment_predictions[0]
            adjusted_predictions[segment_start_idx:segment_end_idx + 1] = segment_predictions + adjustment_at_start

    current_segment_start_idx = segment_end_idx + 1

# 5. Tạo DataFrame kết hợp dữ liệu lịch sử và dự đoán
combined_data = pd.concat([data_day_mbb[data_day_mbb['Date'] <= pd.to_datetime('2025-04-10')], 
                          pd.DataFrame({'Date': future_dates_daily, 'MBB_Price': adjusted_predictions})])
combined_data = combined_data[(combined_data['Date'] >= pd.to_datetime('2020-04-10')) & 
                              (combined_data['Date'] <= pd.to_datetime('2028-05-01'))]
combined_data = combined_data.reset_index(drop=True)

# Lưu kết quả vào file CSV
combined_data.to_csv('MBB_Full_Predictions_2020_2028.csv', index=False)
print("Dữ liệu đầy đủ từ 10/4/2020 đến 1/5/2028 đã được lưu vào 'MBB_Full_Predictions_2020_2028.csv'")

# Biểu đồ tổng quan từ 10/4/2020 đến 1/5/2028
plt.figure(figsize=(14, 7))
plt.plot(data_month_mbb['Date'], data_month_mbb['MBB_Price'], label='Dữ liệu lịch sử (Tháng)', color='blue')
plt.plot(data_day_mbb['Date'], data_day_mbb['MBB_Price'], label='Dữ liệu lịch sử (Ngày)', color='lightblue', alpha=0.5)

predicted_monthly_mbb_filtered = predicted_monthly_mbb[predicted_monthly_mbb['Date'] >= pd.to_datetime('2025-04-10')]
plt.plot(predicted_monthly_mbb_filtered['Date'], predicted_monthly_mbb_filtered['Predicted_Price'], 
         label='Dự đoán (Tháng)', color='orange', marker='o')

plt.plot(combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['Date'], 
         combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['MBB_Price'], 
         label='Dự đoán (Ngày, đã scale)', color='green', linestyle='--')
plt.title('Dự đoán MBB: Tháng và Ngày (Scale liên tục)')
plt.xlabel('Ngày')
plt.ylabel('Giá MBB')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('MBB_Prediction_Chart_Full.png')
plt.close()

# Biểu đồ chi tiết từ 10/4/2025 đến 1/5/2028
mask_detail = (combined_data['Date'] >= pd.to_datetime('2025-04-10')) & (combined_data['Date'] <= pd.to_datetime('2028-05-01'))
combined_data_detail = combined_data[mask_detail]
month_df_detail = predicted_monthly_mbb[
    (predicted_monthly_mbb['Date'] >= pd.to_datetime('2025-04-10')) & 
    (predicted_monthly_mbb['Date'] <= pd.to_datetime('2028-05-01'))
]

plt.figure(figsize=(14, 7))
plt.plot(month_df_detail['Date'], month_df_detail['Predicted_Price'], label='Dự đoán (Tháng)', color='orange', marker='o')
plt.plot(combined_data_detail['Date'], combined_data_detail['MBB_Price'], label='Dự đoán (Ngày, đã scale)', color='green', linestyle='--')
plt.title('Dự đoán MBB: Tháng và Ngày (10/4/2025 - 1/5/2028)')
plt.xlabel('Ngày')
plt.ylabel('Giá MBB')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('MBB_Prediction_Chart_Detail_2025_2028.png')
plt.close()

print("Biểu đồ tổng quan đã được lưu vào 'MBB_Prediction_Chart_Full.png'")
print("Biểu đồ chi tiết (10/4/2025 - 1/5/2028) đã được lưu vào 'MBB_Prediction_Chart_Detail_2025_2028.png'")

19:09:13 - cmdstanpy - INFO - Chain [1] start processing


Dữ liệu đã được xử lý và lưu vào 'MBB_Monthly_Processed.csv' và 'MBB_Daily_Processed.csv'


19:09:14 - cmdstanpy - INFO - Chain [1] done processing


Epoch [1/3], Loss: 0.033672
Epoch [2/3], Loss: 0.008676
Epoch [3/3], Loss: 0.004365
Dữ liệu đầy đủ từ 10/4/2020 đến 1/5/2028 đã được lưu vào 'MBB_Full_Predictions_2020_2028.csv'
Biểu đồ tổng quan đã được lưu vào 'MBB_Prediction_Chart_Full.png'
Biểu đồ chi tiết (10/4/2025 - 1/5/2028) đã được lưu vào 'MBB_Prediction_Chart_Detail_2025_2028.png'


In [12]:
import pandas as pd
import numpy as np
from prophet import Prophet
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 1. Xử lý và lưu dữ liệu MBB
# Đọc dữ liệu MBB
data_day_mbb = pd.read_csv('MBB.csv')
data_day_mbb['Ngày'] = pd.to_datetime(data_day_mbb['Ngày'], format='%d/%m/%Y')
data_day_mbb = data_day_mbb[(data_day_mbb['Ngày'] >= pd.to_datetime('2020-04-10')) & 
                            (data_day_mbb['Ngày'] <= pd.to_datetime('2025-04-10'))]
data_day_mbb['Lần cuối'] = data_day_mbb['Lần cuối'].astype(str).str.replace(',', '').astype(float)

# Tạo dữ liệu đầu tháng cho MBB
data_month_mbb = data_day_mbb.groupby(pd.Grouper(key='Ngày', freq='MS'))['Lần cuối'].first().reset_index()
data_month_mbb.columns = ['Date', 'MBB_Price']

# Đọc dữ liệu S&P 500 và VNINDEX
sp500_data = pd.read_csv('SP500_Full_Predictions_2015_2028.csv')
sp500_data['Date'] = pd.to_datetime(sp500_data['Date'])
vnindex_data = pd.read_csv('VNIINDEX_Full_Predictions_2015_2028.csv')
vnindex_data['Date'] = pd.to_datetime(vnindex_data['Date'])

# Tạo dữ liệu đầu tháng cho S&P 500 và VNINDEX
sp500_month = sp500_data.groupby(pd.Grouper(key='Date', freq='MS'))['Price'].first().reset_index()
vnindex_month = vnindex_data.groupby(pd.Grouper(key='Date', freq='MS'))['Lần cuối'].first().reset_index()

# Ghép dữ liệu tháng
data_month_mbb = data_month_mbb.merge(sp500_month, on='Date', how='left')
data_month_mbb = data_month_mbb.merge(vnindex_month, on='Date', how='left')
data_month_mbb = data_month_mbb.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
data_month_mbb = data_month_mbb.dropna()

# Ghép dữ liệu ngày
data_day_mbb = data_day_mbb.rename(columns={'Ngày': 'Date', 'Lần cuối': 'MBB_Price'})
data_day_mbb = data_day_mbb.merge(sp500_data[['Date', 'Price']], on='Date', how='left')
data_day_mbb = data_day_mbb.merge(vnindex_data[['Date', 'Lần cuối']], on='Date', how='left')
data_day_mbb = data_day_mbb.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
data_day_mbb = data_day_mbb.dropna()

# Lưu dữ liệu đã xử lý
data_month_mbb.to_csv('MBB_Monthly_Processed.csv', index=False)
data_day_mbb.to_csv('MBB_Daily_Processed.csv', index=False)
print("Dữ liệu đã được xử lý và lưu vào 'MBB_Monthly_Processed.csv' và 'MBB_Daily_Processed.csv'")

# 2. Huấn luyện mô hình Prophet để dự đoán giá đầu tháng của MBB
model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
model_prophet.add_regressor('SP500_Price')
model_prophet.add_regressor('VNINDEX_Price')
model_prophet.fit(data_month_mbb.rename(columns={'Date': 'ds', 'MBB_Price': 'y'}))

# Tạo khung thời gian tương lai (từ 1/5/2025 đến 1/5/2028)
future_dates_month = pd.date_range(start='2025-05-01', end='2028-05-01', freq='MS')
future_df_month = pd.DataFrame({'ds': future_dates_month})

# Ghép giá dự đoán đầu tháng của S&P 500 và VNINDEX
future_df_month = future_df_month.merge(sp500_month[sp500_month['Date'] >= '2025-05-01'], 
                                       left_on='ds', right_on='Date', how='left')
future_df_month = future_df_month.merge(vnindex_month[vnindex_month['Date'] >= '2025-05-01'], 
                                       left_on='ds', right_on='Date', how='left')
future_df_month = future_df_month.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
future_df_month = future_df_month[['ds', 'SP500_Price', 'VNINDEX_Price']].ffill()

# Dự đoán giá đầu tháng của MBB
forecast = model_prophet.predict(future_df_month)
predicted_monthly_mbb = forecast[['ds', 'yhat']]
predicted_monthly_mbb.columns = ['Date', 'Predicted_Price']

# 3. Huấn luyện mô hình LSTM để dự đoán giá ngày
scaler = MinMaxScaler()
scaled_data_day = scaler.fit_transform(data_day_mbb[['MBB_Price', 'SP500_Price', 'VNINDEX_Price']])
if np.any(np.isnan(scaled_data_day)) or np.any(np.isinf(scaled_data_day)):
    scaled_data_day = np.nan_to_num(scaled_data_day, nan=0.0, posinf=0.0, neginf=0.0)

time_step = 300
def create_lstm_data(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X, y = create_lstm_data(scaled_data_day, time_step)

X_train_tensor = torch.FloatTensor(X)
y_train_tensor = torch.FloatTensor(y).reshape(-1, 1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, input_size=3, hidden_size=50, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model_lstm = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_lstm = model_lstm.to(device)

num_epochs = 30
for epoch in range(num_epochs):
    model_lstm.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model_lstm(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model_lstm.parameters(), max_norm=1.0)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

# 4. Dự đoán giá ngày từ 10/4/2025 đến 1/5/2028 với scale liên tục
historical_data = scaled_data_day.tolist()
future_dates_daily = pd.date_range(start='2025-04-10', end='2028-05-01', freq='D')
daily_df = pd.DataFrame({'Date': future_dates_daily})

# Ghép giá dự đoán S&P 500 và VNINDEX cho ngày tương lai
daily_df = daily_df.merge(sp500_data[['Date', 'Price']], on='Date', how='left')
daily_df = daily_df.merge(vnindex_data[['Date', 'Lần cuối']], on='Date', how='left')
daily_df = daily_df.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
daily_df = daily_df[['Date', 'SP500_Price', 'VNINDEX_Price']].ffill()

# Chuẩn hóa dữ liệu dự đoán (thêm cột placeholder cho MBB_Price)
daily_df['MBB_Price'] = 0  # Placeholder sẽ được cập nhật sau
scaled_future_data = scaler.transform(daily_df[['MBB_Price', 'SP500_Price', 'VNINDEX_Price']])

# Kiểm tra và bù/cắt dữ liệu để khớp với future_dates_daily
if len(scaled_future_data) > len(future_dates_daily):
    scaled_future_data = scaled_future_data[:len(future_dates_daily)]
elif len(scaled_future_data) < len(future_dates_daily):
    # Bù thêm bằng giá trị cuối cùng nếu thiếu
    last_row = scaled_future_data[-1]
    additional_rows = np.tile(last_row, (len(future_dates_daily) - len(scaled_future_data), 1))
    scaled_future_data = np.vstack((scaled_future_data, additional_rows))

model_lstm.eval()
predictions_daily = []
current_window = historical_data[-time_step:]

# Lấy giá thực tế tại 10/4/2025
last_actual_price = data_day_mbb[data_day_mbb['Date'] == pd.to_datetime('2025-04-10')]['MBB_Price'].iloc[0]

# Dự đoán từng ngày
total_days = len(future_dates_daily)
for i in range(total_days):
    current_window_tensor = torch.FloatTensor(current_window[-time_step:]).unsqueeze(0).to(device)
    with torch.no_grad():
        pred = model_lstm(current_window_tensor).cpu().numpy()[0, 0]
        predictions_daily.append(pred)
        # Cập nhật cửa sổ với giá dự đoán
        new_window_data = np.append(current_window_tensor.cpu().numpy()[0, 1:, :], 
                                   [[pred, scaled_future_data[i, 1], scaled_future_data[i, 2]]], axis=0)
        current_window.append([pred, scaled_future_data[i, 1], scaled_future_data[i, 2]])

# Đảo ngược chuẩn hóa toàn bộ dự đoán
predictions_daily = np.array(predictions_daily).reshape(-1, 1)
predictions_rescaled = scaler.inverse_transform(
    np.hstack((predictions_daily, scaled_future_data[:len(predictions_daily), 1:]))
)[:, 0]

# Scale giá ngày để khớp với giá đầu tháng
adjusted_predictions = predictions_rescaled.copy()
current_segment_start_idx = 0
for j in range(len(predicted_monthly_mbb)):
    if j == 0:
        start_date = pd.to_datetime('2025-04-10')
        end_date = pd.to_datetime('2025-04-30')
        next_month_start = predicted_monthly_mbb['Date'].iloc[j]
    else:
        start_date = predicted_monthly_mbb['Date'].iloc[j - 1]
        end_date = predicted_monthly_mbb['Date'].iloc[j] - pd.Timedelta(days=1)
        next_month_start = predicted_monthly_mbb['Date'].iloc[j]

    if j == len(predicted_monthly_mbb) - 1:
        end_date = pd.to_datetime('2028-05-01')

    if start_date > end_date:
        continue

    segment_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    segment_indices = [i for i, date in enumerate(future_dates_daily) if date in segment_dates]
    if not segment_indices:
        continue

    segment_start_idx = segment_indices[0]
    segment_end_idx = segment_indices[-1]
    segment_predictions = adjusted_predictions[segment_start_idx:segment_end_idx + 1]

    if j == 0:  # Đoạn đầu tiên từ 10/4/2025 đến 30/4/2025
        monthly_price_next = predicted_monthly_mbb[predicted_monthly_mbb['Date'] == next_month_start]['Predicted_Price'].iloc[0]
        daily_price_at_start = last_actual_price  # Giá thực tế tại 10/4/2025
        daily_price_at_end = segment_predictions[-1]

        adjustment_at_start = 0  # Không điều chỉnh giá tại 10/4/2025
        adjustment_at_end = monthly_price_next - daily_price_at_end
        segment_length = len(segment_predictions)
        adjustments = np.linspace(adjustment_at_start, adjustment_at_end, segment_length)
        adjusted_predictions[segment_start_idx:segment_end_idx + 1] = segment_predictions + adjustments
    else:  # Các đoạn sau từ 1/5/2025 trở đi
        monthly_price_current = predicted_monthly_mbb[predicted_monthly_mbb['Date'] == start_date]['Predicted_Price'].iloc[0]
        if end_date >= next_month_start - pd.Timedelta(days=1):
            monthly_price_next = predicted_monthly_mbb[predicted_monthly_mbb['Date'] == next_month_start]['Predicted_Price'].iloc[0]
            daily_price_at_end = segment_predictions[-1]
            daily_price_at_start = segment_predictions[0]

            adjustment_at_start = monthly_price_current - daily_price_at_start
            adjustment_at_end = monthly_price_next - daily_price_at_end
            segment_length = len(segment_predictions)
            adjustments = np.linspace(adjustment_at_start, adjustment_at_end, segment_length)
            adjusted_predictions[segment_start_idx:segment_end_idx + 1] = segment_predictions + adjustments
        else:
            adjustment_at_start = monthly_price_current - segment_predictions[0]
            adjusted_predictions[segment_start_idx:segment_end_idx + 1] = segment_predictions + adjustment_at_start

    current_segment_start_idx = segment_end_idx + 1

# 5. Tạo DataFrame kết hợp dữ liệu lịch sử và dự đoán
combined_data = pd.concat([data_day_mbb[data_day_mbb['Date'] <= pd.to_datetime('2025-04-10')], 
                          pd.DataFrame({'Date': future_dates_daily, 'MBB_Price': adjusted_predictions})])
combined_data = combined_data[(combined_data['Date'] >= pd.to_datetime('2020-04-10')) & 
                              (combined_data['Date'] <= pd.to_datetime('2028-05-01'))]
combined_data = combined_data.reset_index(drop=True)

# Lưu kết quả vào file CSV
combined_data.to_csv('MBB_Full_Predictions_2020_2028.csv', index=False)
print("Dữ liệu đầy đủ từ 10/4/2020 đến 1/5/2028 đã được lưu vào 'MBB_Full_Predictions_2020_2028.csv'")

# Biểu đồ tổng quan từ 10/4/2020 đến 1/5/2028
plt.figure(figsize=(14, 7))
plt.plot(data_month_mbb['Date'], data_month_mbb['MBB_Price'], label='Dữ liệu lịch sử (Tháng)', color='blue')
plt.plot(data_day_mbb['Date'], data_day_mbb['MBB_Price'], label='Dữ liệu lịch sử (Ngày)', color='lightblue', alpha=0.5)

predicted_monthly_mbb_filtered = predicted_monthly_mbb[predicted_monthly_mbb['Date'] >= pd.to_datetime('2025-04-10')]
plt.plot(predicted_monthly_mbb_filtered['Date'], predicted_monthly_mbb_filtered['Predicted_Price'], 
         label='Dự đoán (Tháng)', color='orange', marker='o')

plt.plot(combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['Date'], 
         combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['MBB_Price'], 
         label='Dự đoán (Ngày, đã scale)', color='green', linestyle='--')
plt.title('Dự đoán MBB: Tháng và Ngày (Scale liên tục)')
plt.xlabel('Ngày')
plt.ylabel('Giá MBB')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('MBB_Prediction_Chart_Full.png')
plt.close()

# Biểu đồ chi tiết từ 10/4/2025 đến 1/5/2028
mask_detail = (combined_data['Date'] >= pd.to_datetime('2025-04-10')) & (combined_data['Date'] <= pd.to_datetime('2028-05-01'))
combined_data_detail = combined_data[mask_detail]
month_df_detail = predicted_monthly_mbb[
    (predicted_monthly_mbb['Date'] >= pd.to_datetime('2025-04-10')) & 
    (predicted_monthly_mbb['Date'] <= pd.to_datetime('2028-05-01'))
]

plt.figure(figsize=(14, 7))
plt.plot(month_df_detail['Date'], month_df_detail['Predicted_Price'], label='Dự đoán (Tháng)', color='orange', marker='o')
plt.plot(combined_data_detail['Date'], combined_data_detail['MBB_Price'], label='Dự đoán (Ngày, đã scale)', color='green', linestyle='--')
plt.title('Dự đoán MBB: Tháng và Ngày (10/4/2025 - 1/5/2028)')
plt.xlabel('Ngày')
plt.ylabel('Giá MBB')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('MBB_Prediction_Chart_Detail_2025_2028.png')
plt.close()

print("Biểu đồ tổng quan đã được lưu vào 'MBB_Prediction_Chart_Full.png'")
print("Biểu đồ chi tiết (10/4/2025 - 1/5/2028) đã được lưu vào 'MBB_Prediction_Chart_Detail_2025_2028.png'")

19:54:58 - cmdstanpy - INFO - Chain [1] start processing


Dữ liệu đã được xử lý và lưu vào 'MBB_Monthly_Processed.csv' và 'MBB_Daily_Processed.csv'


19:54:58 - cmdstanpy - INFO - Chain [1] done processing


Epoch [1/30], Loss: 0.019827
Epoch [2/30], Loss: 0.014979
Epoch [3/30], Loss: 0.005650
Epoch [4/30], Loss: 0.002974
Epoch [5/30], Loss: 0.003147
Epoch [6/30], Loss: 0.003062
Epoch [7/30], Loss: 0.002120
Epoch [8/30], Loss: 0.002963
Epoch [9/30], Loss: 0.005250
Epoch [10/30], Loss: 0.003098
Epoch [11/30], Loss: 0.003378
Epoch [12/30], Loss: 0.002011
Epoch [13/30], Loss: 0.003312
Epoch [14/30], Loss: 0.001739
Epoch [15/30], Loss: 0.001384
Epoch [16/30], Loss: 0.001824
Epoch [17/30], Loss: 0.002598
Epoch [18/30], Loss: 0.002609
Epoch [19/30], Loss: 0.001156
Epoch [20/30], Loss: 0.000844
Epoch [21/30], Loss: 0.002267
Epoch [22/30], Loss: 0.000755
Epoch [23/30], Loss: 0.001082
Epoch [24/30], Loss: 0.000711
Epoch [25/30], Loss: 0.000409
Epoch [26/30], Loss: 0.002224
Epoch [27/30], Loss: 0.001716
Epoch [28/30], Loss: 0.002366
Epoch [29/30], Loss: 0.001865
Epoch [30/30], Loss: 0.002302
Dữ liệu đầy đủ từ 10/4/2020 đến 1/5/2028 đã được lưu vào 'MBB_Full_Predictions_2020_2028.csv'
Biểu đồ tổng qu

In [10]:
import pandas as pd
import numpy as np
from prophet import Prophet
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# 1. Xử lý và lưu dữ liệu MBB
# Đọc dữ liệu MBB
data_day_mbb = pd.read_csv('MBB.csv')
data_day_mbb['Ngày'] = pd.to_datetime(data_day_mbb['Ngày'], format='%d/%m/%Y')
data_day_mbb = data_day_mbb[(data_day_mbb['Ngày'] >= pd.to_datetime('2020-04-10')) & 
                            (data_day_mbb['Ngày'] <= pd.to_datetime('2025-04-10'))]
data_day_mbb['Lần cuối'] = data_day_mbb['Lần cuối'].astype(str).str.replace(',', '').astype(float)

# Tạo dữ liệu đầu tháng cho MBB
data_month_mbb = data_day_mbb.groupby(pd.Grouper(key='Ngày', freq='MS'))['Lần cuối'].first().reset_index()
data_month_mbb.columns = ['Date', 'MBB_Price']

# Đọc dữ liệu S&P 500 và VNINDEX
sp500_data = pd.read_csv('SP500_Full_Predictions_2015_2028.csv')
sp500_data['Date'] = pd.to_datetime(sp500_data['Date'])
vnindex_data = pd.read_csv('VNIINDEX_Full_Predictions_2015_2028.csv')
vnindex_data['Date'] = pd.to_datetime(vnindex_data['Date'])

# Tạo dữ liệu đầu tháng cho S&P 500 và VNINDEX
sp500_month = sp500_data.groupby(pd.Grouper(key='Date', freq='MS'))['Price'].first().reset_index()
vnindex_month = vnindex_data.groupby(pd.Grouper(key='Date', freq='MS'))['Lần cuối'].first().reset_index()

# Ghép dữ liệu tháng
data_month_mbb = data_month_mbb.merge(sp500_month, on='Date', how='left')
data_month_mbb = data_month_mbb.merge(vnindex_month, on='Date', how='left')
data_month_mbb = data_month_mbb.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
data_month_mbb = data_month_mbb.dropna()

# Ghép dữ liệu ngày
data_day_mbb = data_day_mbb.rename(columns={'Ngày': 'Date', 'Lần cuối': 'MBB_Price'})
data_day_mbb = data_day_mbb.merge(sp500_data[['Date', 'Price']], on='Date', how='left')
data_day_mbb = data_day_mbb.merge(vnindex_data[['Date', 'Lần cuối']], on='Date', how='left')
data_day_mbb = data_day_mbb.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
data_day_mbb = data_day_mbb.dropna()

# Lưu dữ liệu đã xử lý
data_month_mbb.to_csv('MBB_Monthly_Processed.csv', index=False)
data_day_mbb.to_csv('MBB_Daily_Processed.csv', index=False)
print("Dữ liệu đã được xử lý và lưu vào 'MBB_Monthly_Processed.csv' và 'MBB_Daily_Processed.csv'")

# 2. Huấn luyện mô hình Prophet để dự đoán giá đầu tháng của MBB
model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
model_prophet.add_regressor('SP500_Price')
model_prophet.add_regressor('VNINDEX_Price')
model_prophet.fit(data_month_mbb.rename(columns={'Date': 'ds', 'MBB_Price': 'y'}))

# Tạo khung thời gian tương lai (từ 1/5/2025 đến 1/5/2028)
future_dates_month = pd.date_range(start='2025-05-01', end='2028-05-01', freq='MS')
future_df_month = pd.DataFrame({'ds': future_dates_month})

# Ghép giá dự đoán đầu tháng của S&P 500 và VNINDEX
future_df_month = future_df_month.merge(sp500_month[sp500_month['Date'] >= '2025-05-01'], 
                                       left_on='ds', right_on='Date', how='left')
future_df_month = future_df_month.merge(vnindex_month[vnindex_month['Date'] >= '2025-05-01'], 
                                       left_on='ds', right_on='Date', how='left')
future_df_month = future_df_month.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
future_df_month = future_df_month[['ds', 'SP500_Price', 'VNINDEX_Price']].fillna(method='ffill')

# Dự đoán giá đầu tháng của MBB
forecast = model_prophet.predict(future_df_month)
predicted_monthly_mbb = forecast[['ds', 'yhat']]
predicted_monthly_mbb.columns = ['Date', 'Predicted_Price']

# 3. Huấn luyện mô hình LSTM để dự đoán giá ngày
scaler = MinMaxScaler()
scaled_data_day = scaler.fit_transform(data_day_mbb[['MBB_Price', 'SP500_Price', 'VNINDEX_Price']])
if np.any(np.isnan(scaled_data_day)) or np.any(np.isinf(scaled_data_day)):
    scaled_data_day = np.nan_to_num(scaled_data_day, nan=0.0, posinf=0.0, neginf=0.0)

time_step = 300
def create_lstm_data(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X, y = create_lstm_data(scaled_data_day, time_step)

X_train_tensor = torch.FloatTensor(X)
y_train_tensor = torch.FloatTensor(y).reshape(-1, 1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, input_size=3, hidden_size=50, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model_lstm = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_lstm = model_lstm.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model_lstm.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model_lstm(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        torch.nn.utils.clip_grad_norm_(model_lstm.parameters(), max_norm=1.0)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

# 4. Dự đoán giá ngày từ 10/4/2025 đến 1/5/2028 với scale liên tục
historical_data = scaled_data_day.tolist()
future_dates_daily = pd.date_range(start='2025-04-10', end='2028-05-01', freq='D')
daily_df = pd.DataFrame({'Date': future_dates_daily})

# Ghép giá dự đoán S&P 500 và VNINDEX cho ngày tương lai
daily_df = daily_df.merge(sp500_data[['Date', 'Price']], on='Date', how='left')
daily_df = daily_df.merge(vnindex_data[['Date', 'Lần cuối']], on='Date', how='left')
daily_df = daily_df.rename(columns={'Price': 'SP500_Price', 'Lần cuối': 'VNINDEX_Price'})
daily_df = daily_df[['Date', 'SP500_Price', 'VNINDEX_Price']].fillna(method='ffill')

# Chuẩn hóa dữ liệu dự đoán (thêm cột placeholder cho MBB_Price)
daily_df['MBB_Price'] = 0  # Placeholder sẽ được cập nhật sau
scaled_future_data = scaler.transform(daily_df[['MBB_Price', 'SP500_Price', 'VNINDEX_Price']])

model_lstm.eval()
predictions_daily = []
current_window = historical_data[-time_step:]

# Lấy giá thực tế tại 10/4/2025
last_actual_price = data_day_mbb[data_day_mbb['Date'] == pd.to_datetime('2025-04-10')]['MBB_Price'].iloc[0]

i = 0
for j in range(len(predicted_monthly_mbb)):
    # Đoạn đầu tiên từ 10/4/2025 đến 30/4/2025
    if j == 0:
        start_date = pd.to_datetime('2025-04-10')
        end_date = pd.to_datetime('2025-04-30')
        next_month_start = predicted_monthly_mbb['Date'].iloc[j]
    else:
        start_date = predicted_monthly_mbb['Date'].iloc[j - 1]
        end_date = predicted_monthly_mbb['Date'].iloc[j] - pd.Timedelta(days=1)
        next_month_start = predicted_monthly_mbb['Date'].iloc[j]

    if j == len(predicted_monthly_mbb) - 1:  # Đoạn cuối cùng đến 1/5/2028
        end_date = pd.to_datetime('2028-05-01')

    if start_date > end_date:
        continue

    segment_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    n_days = len(segment_dates)
    if n_days == 0:
        continue

    segment_predictions = []
    current_window_tensor = torch.FloatTensor(current_window[-time_step:]).unsqueeze(0).to(device)
    with torch.no_grad():
        window = current_window_tensor
        for day in range(n_days):
            pred = model_lstm(window).cpu().numpy()[0, 0]
            segment_predictions.append(pred)
            new_window_data = np.append(window.cpu().numpy()[0, 1:, :], 
                                       [[pred, scaled_future_data[i + day, 1], scaled_future_data[i + day, 2]]], axis=0)
            window = torch.FloatTensor(new_window_data).unsqueeze(0).to(device)

    i += n_days

    # Đảo ngược chuẩn hóa
    segment_predictions = np.array(segment_predictions).reshape(-1, 1)
    segment_predictions_rescaled = scaler.inverse_transform(
        np.hstack((segment_predictions, scaled_future_data[i - n_days:i, 1:]))
    )[:, 0]

    # Scale giá ngày để khớp với giá đầu tháng
    if j == 0:  # Đoạn đầu tiên từ 10/4/2025 đến 30/4/2025
        monthly_price_next = predicted_monthly_mbb[predicted_monthly_mbb['Date'] == next_month_start]['Predicted_Price'].iloc[0]
        daily_price_at_start = last_actual_price  # Giá thực tế tại 10/4/2025
        daily_price_at_end = segment_predictions_rescaled[-1]

        adjustment_at_start = 0  # Không điều chỉnh giá tại 10/4/2025, giữ nguyên giá thực tế
        adjustment_at_end = monthly_price_next - daily_price_at_end
        segment_length = n_days
        adjustments = np.linspace(adjustment_at_start, adjustment_at_end, segment_length)
        segment_predictions_scaled = segment_predictions_rescaled + adjustments
    else:  # Các đoạn sau từ 1/5/2025 trở đi
        monthly_price_current = predicted_monthly_mbb[predicted_monthly_mbb['Date'] == start_date]['Predicted_Price'].iloc[0]
        if end_date >= next_month_start - pd.Timedelta(days=1):
            monthly_price_next = predicted_monthly_mbb[predicted_monthly_mbb['Date'] == next_month_start]['Predicted_Price'].iloc[0]
            daily_price_at_end = segment_predictions_rescaled[-1]
            daily_price_at_start = segment_predictions_rescaled[0]

            adjustment_at_start = monthly_price_current - daily_price_at_start
            adjustment_at_end = monthly_price_next - daily_price_at_end
            segment_length = n_days
            adjustments = np.linspace(adjustment_at_start, adjustment_at_end, segment_length)
            segment_predictions_scaled = segment_predictions_rescaled + adjustments
        else:
            adjustment_at_start = monthly_price_current - segment_predictions_rescaled[0]
            segment_predictions_scaled = segment_predictions_rescaled + adjustment_at_start

    # Cập nhật current_window
    scaled_segment = scaler.transform(
        np.column_stack((segment_predictions_scaled.reshape(-1, 1), scaled_future_data[i - n_days:i, 1:]))
    )
    current_window.extend(scaled_segment.tolist())
    predictions_daily.extend(segment_predictions_scaled.tolist())

# 5. Tạo DataFrame kết hợp dữ liệu lịch sử và dự đoán
combined_data = pd.concat([data_day_mbb[data_day_mbb['Date'] <= pd.to_datetime('2025-04-10')], 
                          pd.DataFrame({'Date': future_dates_daily, 'MBB_Price': predictions_daily})])
combined_data = combined_data[(combined_data['Date'] >= pd.to_datetime('2020-04-10')) & 
                              (combined_data['Date'] <= pd.to_datetime('2028-05-01'))]
combined_data = combined_data.reset_index(drop=True)

# Lưu kết quả vào file CSV
combined_data.to_csv('MBB_Full_Predictions_2020_2028.csv', index=False)
print("Dữ liệu đầy đủ từ 10/4/2020 đến 1/5/2028 đã được lưu vào 'MBB_Full_Predictions_2020_2028.csv'")

# Biểu đồ tổng quan từ 10/4/2020 đến 1/5/2028
plt.figure(figsize=(14, 7))
plt.plot(data_month_mbb['Date'], data_month_mbb['MBB_Price'], label='Dữ liệu lịch sử (Tháng)', color='blue')
plt.plot(data_day_mbb['Date'], data_day_mbb['MBB_Price'], label='Dữ liệu lịch sử (Ngày)', color='lightblue', alpha=0.5)

predicted_monthly_mbb_filtered = predicted_monthly_mbb[predicted_monthly_mbb['Date'] >= pd.to_datetime('2025-04-10')]
plt.plot(predicted_monthly_mbb_filtered['Date'], predicted_monthly_mbb_filtered['Predicted_Price'], 
         label='Dự đoán (Tháng)', color='orange', marker='o')

plt.plot(combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['Date'], 
         combined_data[combined_data['Date'] >= pd.to_datetime('2025-04-10')]['MBB_Price'], 
         label='Dự đoán (Ngày, đã scale)', color='green', linestyle='--')
plt.title('Dự đoán MBB: Tháng và Ngày (Scale liên tục)')
plt.xlabel('Ngày')
plt.ylabel('Giá MBB')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('MBB_Prediction_Chart_Full.png')
plt.close()

# Biểu đồ chi tiết từ 10/4/2025 đến 1/5/2028
mask_detail = (combined_data['Date'] >= pd.to_datetime('2025-04-10')) & (combined_data['Date'] <= pd.to_datetime('2028-05-01'))
combined_data_detail = combined_data[mask_detail]
month_df_detail = predicted_monthly_mbb[
    (predicted_monthly_mbb['Date'] >= pd.to_datetime('2025-04-10')) & 
    (predicted_monthly_mbb['Date'] <= pd.to_datetime('2028-05-01'))
]

plt.figure(figsize=(14, 7))
plt.plot(month_df_detail['Date'], month_df_detail['Predicted_Price'], label='Dự đoán (Tháng)', color='orange', marker='o')
plt.plot(combined_data_detail['Date'], combined_data_detail['MBB_Price'], label='Dự đoán (Ngày, đã scale)', color='green', linestyle='--')
plt.title('Dự đoán MBB: Tháng và Ngày (10/4/2025 - 1/5/2028)')
plt.xlabel('Ngày')
plt.ylabel('Giá MBB')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('MBB_Prediction_Chart_Detail_2025_2028.png')
plt.close()

print("Biểu đồ tổng quan đã được lưu vào 'MBB_Prediction_Chart_Full.png'")
print("Biểu đồ chi tiết (10/4/2025 - 1/5/2028) đã được lưu vào 'MBB_Prediction_Chart_Detail_2025_2028.png'")

19:50:15 - cmdstanpy - INFO - Chain [1] start processing


Dữ liệu đã được xử lý và lưu vào 'MBB_Monthly_Processed.csv' và 'MBB_Daily_Processed.csv'


19:50:16 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Tuan Anh\AppData\Local\Temp\ipykernel_23216\1458619213.py:66: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  future_df_month = future_df_month[['ds', 'SP500_Price', 'VNINDEX_Price']].fillna(method='ffill')


Epoch [1/10], Loss: 0.028237
Epoch [2/10], Loss: 0.027970
Epoch [3/10], Loss: 0.010584
Epoch [4/10], Loss: 0.004685
Epoch [5/10], Loss: 0.002717
Epoch [6/10], Loss: 0.001759
Epoch [7/10], Loss: 0.002831
Epoch [8/10], Loss: 0.001762
Epoch [9/10], Loss: 0.001584
Epoch [10/10], Loss: 0.002312


C:\Users\Tuan Anh\AppData\Local\Temp\ipykernel_23216\1458619213.py:137: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily_df = daily_df[['Date', 'SP500_Price', 'VNINDEX_Price']].fillna(method='ffill')
c:\Users\Tuan Anh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\Tuan Anh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\Tuan Anh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\Tuan Anh\AppData\Local\P

Dữ liệu đầy đủ từ 10/4/2020 đến 1/5/2028 đã được lưu vào 'MBB_Full_Predictions_2020_2028.csv'
Biểu đồ tổng quan đã được lưu vào 'MBB_Prediction_Chart_Full.png'
Biểu đồ chi tiết (10/4/2025 - 1/5/2028) đã được lưu vào 'MBB_Prediction_Chart_Detail_2025_2028.png'
